In [ ]:
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 24.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [1]:
!pip install transformers datasets

In [7]:
!pip install transformers huggingface_hub

In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bn-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bn-en")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/309M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [2]:
import pandas as pd
from datasets import Dataset

def load_data(english_file, bengali_file):
    with open(english_file, 'r', encoding='utf-8') as f:
        english_lines = f.readlines()
    with open(bengali_file, 'r', encoding='utf-8') as f:
        bengali_lines = f.readlines()

    data = {'translation': [{'en': en.strip(), 'bn': bn.strip()} for en, bn in zip(english_lines, bengali_lines)]}
    return Dataset.from_dict(data)

# Assuming 'train.en.txt' and 'train.bn.txt' are your training files
train_dataset = load_data('suparadev_en.txt', 'suparadev_bn.txt')
test_dataset = load_data('suparatest2018_en.txt', 'suparatest2018_bn.txt')


In [3]:
# Check the structure of the dataset
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['translation'],
    num_rows: 500
})
Dataset({
    features: ['translation'],
    num_rows: 500
})


In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.utils.data import DataLoader

# Try a different model. This one is publically available.
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bn-en")
# tokenizer = AutoTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')

# Set the target language for the tokenizer
tokenizer.tgt_lang = 'ben' # Set to Bengali

def preprocess_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['bn'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True, padding=True)
    model_inputs = {k: [v for v in values if v is not None] for k, values in model_inputs.items()}
    return model_inputs

# train_dataset = train_dataset.map(preprocess_function, batched=True)
# test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset.set_transform(preprocess_function)
test_dataset.set_transform(preprocess_function)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [14]:
from transformers import AutoModelForSeq2SeqLM

# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bn-en")

In [15]:
# !pip install accelerate -U

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=4,  # Accumulate gradients
    remove_unused_columns=False # Tell the trainer to remove unused columns.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.718395
2,No log,1.914477
3,No log,1.886111


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[63596]], 'forced_eos_token_id': 0}


TrainOutput(global_step=375, training_loss=2.676357421875, metrics={'train_runtime': 1943.8287, 'train_samples_per_second': 0.772, 'train_steps_per_second': 0.193, 'total_flos': 21127950434304.0, 'train_loss': 2.676357421875, 'epoch': 3.0})

In [16]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 1.886110782623291, 'eval_runtime': 137.6654, 'eval_samples_per_second': 3.632, 'eval_steps_per_second': 3.632, 'epoch': 3.0}


In [17]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[63596]], 'forced_eos_token_id': 0}


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/source.spm',
 './saved_model/target.spm',
 './saved_model/added_tokens.json')

In [29]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the saved model and tokenizer
model_dir = "./saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

def translate_text(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)

    # Generate translation using the model
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

    # Decode the generated text
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Get user input and generate translation
user_input = input("Enter the text to be translated from English to Bengali: ")
print(user_input)
translated_output = translate_text(user_input)
print("Translated text:", translated_output)


Enter the text to be translated from English to Bengali: he
he
Translated text: সে


In [30]:
import zipfile
import os
from google.colab import files

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Path to the folder you want to zip
folder_path = './saved_model'

# Path for the output zip file
output_filename = 'final0.zip'

# Create a zip file
zip_folder(folder_path, output_filename)

# Download the zip file
files.download(output_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>